In [1]:
from sklearn.svm import SVC
import json
import pandas as pd

In [2]:
with open("finalData2.json", "r") as f:
    data = json.load(f)

In [10]:
# each row of DF:
# - restaurant
# - text of all reviews together?
# - columns for each ambiance label, 1 or 0

df = pd.DataFrame(columns=['restaurant', 'reviews', 'price', 'stars', 'touristy', 'classy', 'romantic', 'casual', 
                          'hipster', 'divey', 'intimate', 'trendy', 'upscale'])
all_empty = pd.DataFrame(columns=['restaurant', 'reviews', 'price', 'stars', 'touristy', 'classy', 'romantic', 'casual', 
                          'hipster', 'divey', 'intimate', 'trendy', 'upscale'])

In [11]:
for restaurant, restaurant_dict in data['BOSTON'].items():
    ambiances = restaurant_dict['ambience']
    review_texts = [r['text'] for r in restaurant_dict['reviews']]
    review_stars = [r['stars'] for r in restaurant_dict['reviews']]
    review_combined = '\n'.join(review_texts)
    stars_avg = sum(review_stars)/len(review_stars)
    new_row = dict()
    new_row['restaurant'] = restaurant
    new_row['reviews'] = review_combined
    new_row['price'] = restaurant_dict['price']
    new_row['stars'] = stars_avg
    for label in ['touristy', 'classy', 'romantic', 'casual', 'hipster', 'divey', 'intimate', 'trendy', 'upscale']:
        if label in ambiances:
            new_row[label] = 1
        else:
            new_row[label] = 0
    if len(ambiances) == 0:
        all_empty = all_empty.append(new_row, ignore_index=True)
    else:
        df = df.append(new_row, ignore_index=True)

In [67]:
df.head()

,restaurant,reviews,price,stars,touristy,classy,romantic,casual,hipster,divey,intimate,trendy,upscale
0,Legal Sea Foods,"""If it isn't fresh, it isn't legal, "" is Legal...",3,3.5,1,0,0,0,0,0,0,0,0
1,Amrheins,Being seated within 5 mins upon our arrival an...,2,2.0,0,0,0,1,0,0,0,0,0
2,Fiouna's Persian Fusion Cuisine,This one star review...I don't like writing th...,2,2.5,0,0,0,1,0,0,0,0,0
3,Code 10,The reason why I'm giving this place a 2 at al...,1,3.5,0,0,0,1,0,0,0,0,0
4,Shenannigans Bar,I Will Pay A $20 Cover To Go To This Bar When ...,2,1.5,0,0,0,1,0,0,0,0,0


In [71]:
for col in ['touristy', 'classy', 'romantic', 'casual', 'hipster', 'divey', 'intimate', 'trendy', 'upscale', 'price']:
    df[col] = df[col].astype(int)
    all_empty[col] = all_empty[col].astype(int)

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [73]:
X = df[['reviews', 'stars', 'price']]
Y = df.drop(['restaurant', 'reviews'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
vec = TfidfVectorizer()
Xtrain_features = pd.DataFrame(vec.fit_transform(X_train['reviews']).toarray())
Xtest_features = pd.DataFrame(vec.transform(X_test['reviews']).toarray())

In [74]:
Xtrain_features['stars'] = X_train['stars'].reset_index()['stars']
Xtrain_features['price'] = X_train['price'].reset_index()['price']
Xtest_features['stars'] = X_test['stars'].reset_index()['stars']
Xtest_features['price'] = X_test['price'].reset_index()['price']

In [75]:
df.describe()

,price,stars,touristy,classy,romantic,casual,hipster,divey,intimate,trendy,upscale
count,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000,1357.000000
mean,1.933677,3.470155,0.020634,0.397200,0.042004,0.661017,0.031688,0.034635,0.033161,0.171702,0.041268
std,0.717476,1.103107,0.142207,0.489498,0.200673,0.473539,0.175231,0.182922,0.179124,0.377261,0.198982
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,3.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,4.500000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
df[df['touristy'] == 1].describe()

,price,stars,touristy,classy,romantic,casual,hipster,divey,intimate,trendy,upscale
count,28.000000,28.000000,28.0,28.000000,28.0,28.000000,28.0,28.000000,28.0,28.000000,28.0
mean,1.964286,3.553571,1.0,0.107143,0.0,0.571429,0.0,0.035714,0.0,0.107143,0.0
std,0.507875,1.124963,0.0,0.314970,0.0,0.503953,0.0,0.188982,0.0,0.314970,0.0
min,1.000000,1.000000,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
25%,2.000000,3.000000,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
50%,2.000000,3.500000,1.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.000000,0.0
75%,2.000000,4.500000,1.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.000000,0.0
max,3.000000,5.000000,1.0,1.000000,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.0


In [90]:
svm = SVC()

In [150]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
param_grid = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear']} 
param_grid = {'C': [0.1], 
              'gamma': [1],
              'kernel': ['rbf']} 
balanced_scorer = make_scorer(balanced_accuracy_score)

models = dict()
  
grid = GridSearchCV(svm, param_grid, refit = True, verbose = 3, scoring=balanced_scorer)
for col in ['touristy', 'classy', 'romantic', 'casual', 'hipster', 'divey', 'intimate', 'trendy', 'upscale']:
    grid.fit(Xtrain_features, y_train[col])
    print(grid.best_score_)
    print(grid.best_estimator_)
    model = grid.best_estimator_
    models[col] = model

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.500, total=  30.9s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   30.8s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.500, total=  25.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


...

[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.500, total=  29.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   52.0s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.500, total=  33.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.500, total=  32.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.500, total=  30.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.5min finished


0.5
SVC(C=0.1, gamma=1)


In [151]:
models

{'touristy': SVC(C=0.1, gamma=1),
 'classy': SVC(C=0.1, gamma=1),
 'romantic': SVC(C=0.1, gamma=1),
 'casual': SVC(C=0.1, gamma=1),
 'hipster': SVC(C=0.1, gamma=1),
 'divey': SVC(C=0.1, gamma=1),
 'intimate': SVC(C=0.1, gamma=1),
 'trendy': SVC(C=0.1, gamma=1),
 'upscale': SVC(C=0.1, gamma=1)}

In [159]:
load = pickle.load(open( "test_model", "rb" ) )
preds = load.predict(Xtest_features)

In [91]:
from sklearn.metrics import classification_report, confusion_matrix

In [160]:
print(confusion_matrix(y_test['casual'], preds))

[[ 63  79]
 [  7 259]]


In [161]:
print(classification_report(y_test['casual'], preds))

              precision    recall  f1-score   support

           0       0.90      0.44      0.59       142
           1       0.77      0.97      0.86       266

    accuracy                           0.79       408
   macro avg       0.83      0.71      0.73       408
weighted avg       0.81      0.79      0.77       408



In [158]:
import pickle
pickle.dump(models['casual'], open("test_model", "wb"))

In [81]:
all_empty_features = pd.DataFrame(vec.transform(all_empty['reviews']).toarray())

In [82]:
all_empty_features['stars'] = all_empty['stars'].reset_index()['stars']
all_empty_features['price'] = all_empty['price'].reset_index()['price']

In [83]:
all_empty

,restaurant,reviews,price,stars,touristy,classy,romantic,casual,hipster,divey,intimate,trendy,upscale
0,Espresso Minute,Very good coffee and solid crepes. I would adm...,1,3.0,0,0,0,1,0,0,0,0,0
1,Star Market,Star Market is a grocery chain with their newe...,2,3.0,0,0,0,1,0,0,0,0,0
2,McGann's Pub,My colleague and I tried to find a bite to eat...,2,3.5,0,0,0,1,0,0,0,0,0
3,Burger King,When I leave Boston this July; I will never fo...,1,2.5,0,0,0,1,0,0,0,0,0
4,Campo di Fiori,PUBLIC SERVICE ANNOUNCEMENT...\nCampo is re-op...,3,2.5,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Garifuna Cafe,This is a Honduran restaurant that recently op...,3,3.0,0,0,0,0,0,0,0,0,0
863,Boston Harbor Hotel,"On a return whirlwind trip of New England, I h...",3,5.0,0,0,0,0,0,0,0,0,0
864,Stix Restaurant & Lounge,"Congratulations are in order, yelpers! I have ...",2,3.0,0,0,0,1,0,0,0,0,0
865,Boston Foodie Tours,"Oh, Boston Foodie Tours has a five star rating...",2,4.5,0,0,0,1,0,0,0,0,0


In [84]:
all_empty['classy'] = svm.predict(all_empty_features)

In [131]:
for index, row in all_empty[df['classy'] == 1].iterrows():
    print("Restaurant: " + row['restaurant'])
    print(row['reviews'])
    print("\n")

Restaurant: Al Capone Pizza Pasta & Meats
CHEAP pizza and lots of it. Order a slice and you get a full third of a pizza for under $3 if I remember correctly. It's not the most orgasmic pizza you'll ever have, but it's good and the price makes it even better! There's not much atmosphere and the seating upstairs gets a little stuffy sometimes. If it's nice out, take your pizza down the street and around the corner to the plaza at 100 Summer and enjoy!
Five stars for LOTS of food!  Their "small" subs are about 10", and their large subs is exactly 20".  (Quality is not bad at all, and they don't cost more than usual.)


Restaurant: Floramo's Deli
Huge, flavorful grilled chicken kebab, over a pile of rice pilaf, and with a large side salad. Cost? $6.50.

Order this and be happy. :-)
I just returned from Floramo's Deli and I've gotta tell ya... my stomach is revolting. So why two stars? Here's the deal...

If you are in need of comfort food, but would rather not resort to the pasta route, th

C:\Users\sahar\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
